In [ ]:
!pip install olefile
!pip install docx
!pip install exceptions
!pip install python-pptx
!pip install python-docx
!pip install tika

In [ ]:
# connect google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#hwp, pptx, docx, xls, csv, xlsx. pdf -> txt로 변환

In [ ]:
import olefile
import pandas as pd
import glob
from os import rename, listdir
import os
from datetime import datetime as dt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

In [ ]:
#리스트 -> 문자열
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

In [ ]:
# 정규표현식 한글 남겨두고 특수문자, \r\n 제거하여 관련 정규문
import re

def remove_special_characters(text):
    return re.sub(r'[^0-9a-zA-Z가-힣\s\r\n]', '', text).strip()

In [ ]:
target_path = input("검사하실 target문서의 directory경로를 입력하시요: ")
compare_path = input("문서 뭉치 directory경로를 입력하시오: ")

In [ ]:
#검사하실 target문서 directory 경로
# hwp 확장자 dataFrame으로 변환
filelist_hwp = glob.glob(f'{target_path}/'+'*.hwp')


hwp_df = pd.DataFrame() 
hwp_df['title'] = np.nan
names = [os.path.basename(x) for x in filelist_hwp]
hwp_df['title'] = names


content = []
for file in filelist_hwp :
    f = olefile.OleFileIO(file) #파일열기
    encoded_text = f.openstream('PrvText').read() # PrvText 스트림의 내용 꺼내기
    decoded_text = encoded_text.decode('UTF-16') # 유니코드를 UTF-16으로 디코딩
    content.append(decoded_text)

hwp_df['content'] = np.nan
hwp_df['content'] = content
hwp_df['extension'] = "hwp"

# pptx 확장자 dataFrame으로 변환
from pptx import Presentation
filelist_pptx = glob.glob(f'{target_path}/'+'*.pptx')
pptx_df = pd.DataFrame()
names = [os.path.basename(x) for x in filelist_pptx]
for i in filelist_pptx:
  prs = Presentation(f"{i}")

  

  data = []
  for slide in prs.slides:
    
    for shape in slide.shapes:
        if not shape.has_text_frame:
              continue
        text_frame = shape.text_frame
        for paragraph in text_frame.paragraphs:
              for run in paragraph.runs:

                  data.append(run.text)


  pptx_df['title'] = np.nan
  pptx_df['content'] = np.nan
  pptx_df['extension'] = np.nan
  pptx_df["content"] = [listToString(data)]
  pptx_df["title"] = names          
  pptx_df['extension'] = "pptx"

# docx 확장자 dataFrame으로 변환
import docx
filelist_docx = glob.glob(f'{target_path}/'+'*.docx')
docx_df = pd.DataFrame()
names = [os.path.basename(x) for x in filelist_docx]
docx_df['title'] = names
def extract_text_from_docx(file):
    doc = docx.Document(file)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)
content = []
for file in filelist_docx:
    if file.endswith('.docx'):
        extracted_text = extract_text_from_docx(file)
        content.append(extracted_text)
docx_df['content'] = content
docx_df['extension'] = 'docx'


# pdf 사용시 dataFrame으로 변환
from tika import parser
filelist_pdf = glob.glob(f'{target_path}/'+'*.pdf')
names = [os.path.basename(x) for x in filelist_pdf]
pdf_df = pd.DataFrame()

pdf_df['title'] = np.nan
pdf_df['title'] = listToString(names)

pdf_df['content'] = np.nan
content_list = []
for i in filelist_pdf:
  pdf_path = f"{i}"
  parsed = parser.from_file(pdf_path)
  txt = open('output.txt', 'w', encoding = 'utf-8')
  # output.txt에 pdf파일 내용을 write
  content_list.append(parsed['content'])

  txt.close()
pdf_df = pd.DataFrame([names,content_list])
pdf_df = pdf_df.T
pdf_df = pdf_df.rename(columns={0:'title'})
pdf_df = pdf_df.rename(columns={1:'content'})
pdf_df['extension'] = 'pdf'

# xlsx 사용시 dataFrame으로 변환
filelist_xlsx = glob.glob(f'{target_path}/'+'*.xlsx')
names = [os.path.basename(x) for x in filelist_xlsx]
csv_xlsx_df = pd.DataFrame()


for i in filelist_xlsx:
    csv_xlsx_df = pd.read_excel(f"{i}")
  
full_text = []
for column in csv_xlsx_df:
    for item in csv_xlsx_df[column]:

        full_text.append(str(item))
csv_xlsx_df = csv_xlsx_df.head(1)

csv_xlsx_df['title'] = np.nan
csv_xlsx_df['title'] = listToString(names)
csv_xlsx_df['content'] = listToString(full_text)
csv_xlsx_df['extension'] = 'csv'
csv_xlsx_df = csv_xlsx_df[['title','content','extension']]

In [ ]:
df_target_merged = pd.concat([hwp_df, pptx_df, docx_df,pdf_df,csv_xlsx_df])
df_target_merged = df_target_merged.reset_index()
df_target_merged = df_target_merged.drop(['index'],axis=1)

In [ ]:
for i in range(len(df_target_merged['content'])):
  df_target_merged['content'].loc[i] = remove_special_characters(df_target_merged['content'].loc[i])

In [ ]:
df_target_merged

In [ ]:
#비교 대상 문서 directory 경로
# hwp 확장자 dataFrame으로 변환
filelist_hwp = glob.glob(f'{compare_path}/'+'*.hwp')


hwp_df = pd.DataFrame() 
hwp_df['title'] = np.nan
names = [os.path.basename(x) for x in filelist_hwp]
hwp_df['title'] = names


content = []
for file in filelist_hwp :
    f = olefile.OleFileIO(file) #파일열기
    encoded_text = f.openstream('PrvText').read() # PrvText 스트림의 내용 꺼내기
    decoded_text = encoded_text.decode('UTF-16') # 유니코드를 UTF-16으로 디코딩
    content.append(decoded_text)

hwp_df['content'] = np.nan
hwp_df['content'] = content
hwp_df['extension'] = "hwp"

# pptx 확장자 dataFrame으로 변환
from pptx import Presentation
filelist_pptx = glob.glob(f'{compare_path}/'+'*.pptx')
pptx_df = pd.DataFrame()
names = [os.path.basename(x) for x in filelist_pptx]
for i in filelist_pptx:
  prs = Presentation(f"{i}")

  

  data = []
  for slide in prs.slides:
    
    for shape in slide.shapes:
        if not shape.has_text_frame:
              continue
        text_frame = shape.text_frame
        for paragraph in text_frame.paragraphs:
              for run in paragraph.runs:

                  data.append(run.text)


  pptx_df['title'] = np.nan
  pptx_df['content'] = np.nan
  pptx_df['extension'] = np.nan
  pptx_df["content"] = [listToString(data)]
  pptx_df["title"] = names          
  pptx_df['extension'] = "pptx"

# docx 확장자 dataFrame으로 변환
import docx
filelist_docx = glob.glob(f'{compare_path}/'+'*.docx')
# target = 'target.docx'
# base = 0
docx_df = pd.DataFrame()
names = [os.path.basename(x) for x in filelist_docx]
docx_df['title'] = names
# index = df[df['title'] == target].index[0]
def extract_text_from_docx(file):
    doc = docx.Document(file)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)
content = []
for file in filelist_docx:
    if file.endswith('.docx'):
        extracted_text = extract_text_from_docx(file)
        content.append(extracted_text)
docx_df['content'] = content
docx_df['extension'] = 'docx'

# xls 사용시 dataFrame으로 변환
filelist_xlsx = glob.glob(f'{compare_path}/'+'*.xls')
names = [os.path.basename(x) for x in filelist_xlsx]
csv_df = pd.DataFrame()

full_text = []

for i in filelist_xlsx:
    csv_df = pd.read_csv(f"{i}")

for column in csv_df:
    for item in csv_df[column]:

        full_text.append(str(item))



csv_df['title'] = np.nan
csv_df['title'] = listToString(names)
csv_df['content'] = listToString(full_text)
csv_df['extension'] = 'csv'
csv_df = pd.DataFrame((csv_df[['title','content','extension']].iloc[len(filelist_xlsx) -1])).T

# pdf 사용시 dataFrame으로 변환
from tika import parser
filelist_pdf = glob.glob(f'{compare_path}/'+'*.pdf')
names = [os.path.basename(x) for x in filelist_pdf]
pdf_df = pd.DataFrame()

pdf_df['title'] = np.nan
pdf_df['title'] = listToString(names)

pdf_df['content'] = np.nan
content_list = []
for i in filelist_pdf:
  pdf_path = f"{i}"
  parsed = parser.from_file(pdf_path)
  txt = open('output.txt', 'w', encoding = 'utf-8')
  # output.txt에 pdf파일 내용을 write
  content_list.append(parsed['content'])

  txt.close()
pdf_df = pd.DataFrame([names,content_list])
pdf_df = pdf_df.T
pdf_df = pdf_df.rename(columns={0:'title'})
pdf_df = pdf_df.rename(columns={1:'content'})
pdf_df['extension'] = 'pdf'

# xlsx 사용시 dataFrame으로 변환
filelist_xlsx = glob.glob(f'{compare_path}/'+'*.xlsx')
names = [os.path.basename(x) for x in filelist_xlsx]
csv_xlsx_df = pd.DataFrame()


for i in filelist_xlsx:
    csv_xlsx_df = pd.read_excel(f"{i}")

full_text = []
for column in csv_df:
    for item in csv_df[column]:

        full_text.append(str(item))



csv_xlsx_df['title'] = np.nan
csv_xlsx_df['title'] = listToString(names)
csv_xlsx_df['content'] = listToString(full_text)
csv_xlsx_df['extension'] = 'csv'
csv_xlsx_df = pd.DataFrame((csv_df[['title','content','extension']].iloc[len(filelist_xlsx) -1])).T

In [ ]:
df_compare_merged = pd.concat([hwp_df, docx_df, pptx_df,csv_df,pdf_df,csv_xlsx_df])
df_compare_merged = df_compare_merged.reset_index()
df_compare_merged = df_compare_merged.drop(['index'],axis=1)
for i in range(len(df_compare_merged['content'])):
  df_compare_merged['content'].loc[i] = remove_special_characters(df_compare_merged['content'].loc[i])

In [ ]:
df_compare_merged